In [1]:
import os
from langchain_huggingface import HuggingFaceEndpoint
import pandas as pd
import numpy as np

from langchain import HuggingFaceHub, PromptTemplate, LLMChain
from langchain_core.prompts import PromptTemplate
from langchain.chains import LLMChain
import json

import requests
from tenacity import retry, stop_after_attempt, wait_exponential, retry_if_exception_type
from langchain.llms import HuggingFaceHub
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

from huggingface_hub import InferenceClient
from concurrent.futures import ThreadPoolExecutor, as_completed
import threading
import logging
import pandas as pd
from langchain import PromptTemplate, LLMChain
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [24]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
import logging
import time

model_param = {
    "best_of": 1,
    "decoder_input_details": False,
    "details": False,
    "do_sample": False,
    "frequency_penalty": 0.1,
    "grammar": None,
    "max_new_tokens": 800,
    "repetition_penalty": 1.01,
    "return_full_text": False,
    "seed": 42,
    "temperature": 0.2,
    "top_k": 25,
    "top_n_tokens": 5,
    "top_p": 0.95,
    "truncate": None,
    "typical_p": 0.95,
    "watermark": True
}

# กำหนดโปรมต์เริ่มต้น
default_prompt = """ 
YOU ARE AN EXPERT SUMMARIZER, SKILLED IN CONDENSING LONG THAI TEXTS INTO CONCISE SUMMARIES WHILE PRESERVING THE ORIGINAL SENTENCE STRUCTURE AND FOCUSING ON KEY ISSUES.
###INSTRUCTIONS###

- ALWAYS ANSWER TO THE USER IN THAI.
- SUMMARIZE THE LONG THAI TEXT BY MAINTAINING THE ORIGINAL SENTENCE STRUCTURE.
- FOCUS ON SUMMARIZING KEY POINTS RELATED TO ISSUES OR PROBLEMS MENTIONED IN THE TEXT.
- PRESERVE THE ORIGINAL MEANING AND CONTEXT OF THE TEXT.

###Chain of Thoughts###

1. **Reading Comprehension:**
   1.1. READ THE LONG THAI TEXT CAREFULLY TO FULLY UNDERSTAND ITS CONTENT.
   1.2. IDENTIFY KEY POINTS RELATED TO ISSUES OR PROBLEMS MENTIONED IN THE TEXT.

2. **Summary Generation:**
   2.1. SUMMARIZE THE KEY POINTS RELATED TO ISSUES OR PROBLEMS WHILE MAINTAINING THE ORIGINAL SENTENCE STRUCTURE.
   2.2. PRESERVE THE ORIGINAL MEANING AND CONTEXT OF THE TEXT BY SELECTIVELY REMOVING NON-ESSENTIAL INFORMATION.
   2.3. ENSURE THE SUMMARY IS CONCISE AND FOCUSED ON THE MAIN ISSUES.

3. **Review:**
   3.1. CHECK THE SUMMARY FOR ACCURACY AND COMPLETENESS.
   3.2. ENSURE THE SUMMARY IS FREE OF ERRORS AND MAINTAINS THE ORIGINAL CONTEXT.

###What Not To Do###

OBEY and never do:
- **NEVER** CHANGE THE ORIGINAL SENTENCE STRUCTURE OR MEANING OF THE TEXT.
- **DO NOT** MISS KEY POINTS RELATED TO ISSUES OR PROBLEMS MENTIONED IN THE TEXT.
- **NEVER** INCLUDE NON-ESSENTIAL INFORMATION IN THE SUMMARY.
- **DO NOT** FAIL TO PRESERVE THE ORIGINAL CONTEXT AND MEANING OF THE TEXT.

###Few-Shot Example (never copy it)###
**Input Text:** "การปรับปรุงระบบการจัดการข้อมูลของบริษัทเป็นเรื่องที่สำคัญมาก เนื่องจากข้อมูลที่ไม่ถูกต้องอาจทำให้การดำเนินงานของบริษัทเกิดปัญหาได้ อย่างไรก็ตาม การปรับปรุงระบบการจัดการข้อมูลนี้ต้องใช้เวลาและทรัพยากรมาก ทำให้บริษัทต้องเผชิญกับความท้าทายในการจัดการข้อมูลอย่างมีประสิทธิภาพ"

**Expected Summary:** "การปรับปรุงระบบการจัดการข้อมูลของบริษัทเป็นเรื่องที่สำคัญมาก เพื่อป้องกันปัญหาที่เกิดจากข้อมูลที่ไม่ถูกต้อง แต่การปรับปรุงนี้ต้องใช้เวลาและทรัพยากรมาก ทำให้บริษัทเผชิญกับความท้าทาย"


{}

### THIS IS A MANDATORY DIRECTIVE ###
### FORMAT YOUR RESPONSE AS FOLLOWS:
- **ANSWER:** [SUMMARY]

DO NOT RESPOND WITH ANYTHING ELSE OUTSIDE OF THIS FORMAT.

"""


def read_text_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return file.read()

def split_text_using_recursive(text, chunk_size, chunk_overlap):  
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap
    )
    return text_splitter.split_text(text)  

def generate_prompts(chunks, prompt_template, model_params):
    prompts = []
    for chunk in chunks:
        prompt = prompt_template.format(chunk)
        prompts.append({
            "input": prompt,
            "params": model_params
        })
    return prompts

def generate_summary_responses(prompts, client, retries=3):
    responses = []
    for prompt in prompts:
        for attempt in range(retries):
            try:
                response = client.text_generation(prompt['input'], **prompt['params'])
                responses.append(response)
                break  # Break the retry loop if successful
            except Exception as e:
                logging.error(f"Attempt {attempt + 1} failed: {e}")
                if "Model is overloaded" in str(e):
                    backoff_time = 2 ** attempt  
                    time.sleep(backoff_time)
                else:
                    raise  
        else:
            raise Exception("Max retries exceeded")
    return responses

file_path = r'C:\Users\kantaphong\Desktop\hugging_face\test_llm\Thai_text\cpall_2565.txt'
text = read_text_file(file_path)

chunks = split_text_using_recursive(text, chunk_size=13185, chunk_overlap=1318)
prompts = generate_prompts(chunks, default_prompt, model_param)

client = InferenceClient('https://ai-api.manageai.co.th/llm-model-02/')
responses = generate_summary_responses(prompts, client)

for i, response in enumerate(responses):
    print(f"Response for chunk {i+1}:")
    print(response)
    print("\n--------------------------------------------------\n")

Response for chunk 1:
**ANSWER:** บริษัท CP All (CPR) ได้ดำเนินการตามแผนการสื่อสาร ESG ในไตรมาสที่ 4 ปี 2565 ด้วยการเน้นสื่อสารกับกลุ่มคนรุ่นใหม่อายุไม่เกิน 30 ปี และสร้างการรับรู้ในกลุ่มผู้มีส่วนได้เสียผ่านช่องทางหลัก เช่น โซเชียลมีเดีย, เว็บข่าว, Facebook, TikTok, Twitter, YouTube, Influencer และ Blogger บริษัทได้รับรางวัลและเกียรติคุณต่างๆ ในปี 2565 อาทิ DJSI World ด้าน Food and Staple Retailing, THSI, และ Asia Excellence Awards นอกจากนี้ CPR ยังได้รับรางวัลในด้านการสนับสนุนคนพิการ รับรางวัล Reading ESG 2022 และรับรางวัลในงาน Asia Energy Awards ปี 2562 บริษัทยังได้ดำเนินการต่างๆ ในด้านสังคม, การศึกษา, การอนุรักษ์พลังงาน, การบริหารจัดการขยะ, และการสนับสนุน SMEs ผ่านโครงการและกิจกรรมต่างๆ นอกจากนี้บริษัทยังได้จัดประชุมผู้ถือหุ้นผ่านระบบออนไลน์ เพื่อป้องกันการแพร่ระบาดของโรค COVID-19 และมีการตรวจสอบคะแนนเสียงจากผู้ถือหุ้นผ่านแอปพลิเคชัน IR Plus AGM.

--------------------------------------------------

Response for chunk 2:
**ANSWER:** บริษัทรายงานผลการดำเนินงานในปี 2565 ซึ่งมีรายได้รวม

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
import logging
import time

# กำหนดพารามิเตอร์ของโมเดล
model_param = {
    "best_of": 1,
    "decoder_input_details": False,
    "details": False,
    "do_sample": False,
    "frequency_penalty": 0.1,
    "grammar": None,
    "max_new_tokens": 1000,
    "repetition_penalty": 1.01,
    "return_full_text": False,
    "seed": 42,
    "temperature": 0.1,
    "top_k": 15,
    "top_n_tokens": 5,
    "top_p": 0.95,
    "truncate": None,
    "typical_p": 0.95,
    "watermark": True
}

# กำหนดโปรมต์เริ่มต้น
default_prompt = """ YOU ARE A SKILLED SUMMARIZER, ABLE TO EXTRACT THE MOST IMPORTANT POINTS FROM A THAI TEXT AND PRESENT THEM IN A CLEAR AND CONCISE MANNER. YOU HAVE BEEN TRAINED ON A VARIETY OF TEXTS AND CAN IDENTIFY THE KEY ELEMENTS THAT SUMMARIZE THE MAIN IDEA.

###INSTRUCTIONS###

- READ THE THAI TEXT CAREFULLY AND IDENTIFY THE MAIN TOPIC OR IDEA.
- EXTRACT THE MOST IMPORTANT POINTS THAT SUMMARIZE THE TEXT, OMITTING ANY UNNECESSARY DETAILS.
- PRESENT THE SUMMARY IN A CLEAR AND CONCISE MANNER, USING PROPER THAI LANGUAGE AND GRAMMAR.

###Chain of Thoughts###

1. **Comprehension:**
   1.1. READ THE THAI TEXT TO UNDERSTAND THE MAIN TOPIC OR IDEA.
   1.2. IDENTIFY THE KEY ELEMENTS THAT SUPPORT THE MAIN IDEA.

2. **Summary:**
   2.1. EXTRACT THE MOST IMPORTANT POINTS FROM THE TEXT, OMITTING ANY UNNECESSARY DETAILS.
   2.2. ORGANIZE THE POINTS IN A LOGICAL AND COHERENT MANNER.

3. **Review:**
   3.1. CHECK THE SUMMARY FOR ACCURACY AND COMPLETENESS.
   3.2. ENSURE THE SUMMARY IS CLEAR, CONCISE, AND FREE OF ERRORS.

###What Not To Do###

OBEY and never do:
- **NEVER** INCLUDE UNNECESSARY DETAILS OR EXAMPLES IN THE SUMMARY.
- **DO NOT** OMIT KEY POINTS THAT ARE ESSENTIAL TO UNDERSTANDING THE MAIN IDEA.
- **NEVER** PROVIDE A SUMMARY THAT IS LONGER THAN NECESSARY OR CONTAINS ERRORS.
- **DO NOT** FAIL TO REVIEW THE SUMMARY FOR ACCURACY AND COMPLETENESS.

###Few-Shot Example###
**Input:** [Example Thai text]
**Output:** 
-Main point 1
-Main point 2
-Main point 3
 
**WHEN SUMMARY POINT ALREADY, DO NOT REPLY ANYTHING ADDITIONALLY.**
**THINK STEP BY STEP HOW MANY MAIN POINT SHOULD HAVE IN ONE CHUNK** 
**RESPONSE ONLY **MAIN POINT**

{}

###THIS IS A MANDATORY DIRECTIVE###
### FORMAT ANSWER ### 
-MAIN POINT [NUMBER]
"""

def read_text_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return file.read()

def split_text_using_recursive(text, chunk_size, chunk_overlap):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap
    )
    return text_splitter.split_text(text)  

def dynamic_chunking(text, min_chunk_size, max_chunk_size, max_overlap):
    text_length = len(text)
    
    if text_length < min_chunk_size:
        chunk_size = text_length
        chunk_overlap = 0
    else:
        chunk_size = min(max_chunk_size, text_length // (text_length // max_chunk_size))
        chunk_overlap = min(max_overlap, chunk_size // 10)
    
    return split_text_using_recursive(text, chunk_size, chunk_overlap)

def generate_prompts(chunks, prompt_template, model_params):
    prompts = []
    for chunk in chunks:
        prompt = prompt_template.format(chunk)
        prompts.append({
            "input": prompt,
            "params": model_params
        })
    return prompts

def generate_summary_responses(prompts, client, retries=3):
    responses = []
    for prompt in prompts:
        for attempt in range(retries):
            try:
                response = client.text_generation(prompt['input'], **prompt['params'])
                responses.append(response)
                break  # Break the retry loop if successful
            except Exception as e:
                logging.error(f"Attempt {attempt + 1} failed: {e}")
                if "Model is overloaded" in str(e):
                    backoff_time = 2 ** attempt  
                    time.sleep(backoff_time)
                else:
                    raise  
        else:
            raise Exception("Max retries exceeded")
    return responses

file_path = r'C:\Users\kantaphong\Desktop\hugging_face\test_llm\Thai_text\ไฟล์ข้อความท่องเที่ยว.txt'
text = read_text_file(file_path)

# Dynamic chunking based on text length
min_chunk_size = 5000
max_chunk_size = 14000
max_overlap = 1800

chunks = dynamic_chunking(text, min_chunk_size, max_chunk_size, max_overlap)
prompts = generate_prompts(chunks, default_prompt, model_param)

client = InferenceClient('https://ai-api.manageai.co.th/llm-model-02/')
responses = generate_summary_responses(prompts, client)

for i, response in enumerate(responses):
    print(f"Response for chunk {i+1}:")
    print(response)
    print("\n--------------------------------------------------\n")


ZeroDivisionError: integer division or modulo by zero

In [14]:
chunks[0]

'สนับสนุนโยบายอีกโปรดติดตามวดล้อม และด้านธรรมพิบาลของ CPLกลยุทธ์การสื่อสาร ESG ในไตรมาสที่ 4 ของปี 2565จะเน้นสื่อสารในเข้าถึงกลุ่มคนรุ่นใหม่ อายุต่ำกว่า 30 ปีมากขึ้นและยังคงสร้างการดับรู้ในกลุ่มสเต็คโฮเดอร์สำคัญ ๆ ในทุกกลุ่มผ่านสื่อประแสหลัก โซเชียลมีเดีย ทางเว็บข่าวfacebook tiktok twitter youtube influencer และ bloggerเริ่มต้นข่าวดีดีในไตรมาสที่สี่ djsi to cpr หนึ่งเดียวของไทยใน djsi world กลุ่ม food and staple retailing ปีสองพันที่สิบสองการันติองค์กรยั่งยืนระดับโลก 6 ปีต่อเนื่องCPR ติดไกลชื่อหุ้นย่างยืน THSI ต่อเนื่องเป็นปีที่ 5CPR คว้า 5 รางวัลจากเวที Asia Excellence Award ครั้งที่ 12CPR ต่อย้ำการทำงานตามแนวทางพัฒนาย่างย่างยืนรับรางวัล Reading ESG 2022CPR รับรางวัลองค์กรสำหรับสนุนคนพิการดีเด่น 6 ปีต่อเนื่องApex 2022 จับตามองเวทีแห่งโอกาส 7-11 ร่วมใจคนไทยเป็นเจ้าบ้านที่ดี ชูนวัตกรรม สู่ความยั่งยืนเอเชียนเอเนอร์จี อาวอร์ด สองพันยี่สิบสอง มอบรางวัลอาคารสร้างสรรค์เพื่อการอนุรักษ์พลังงานให้กับ สถาบัติการจัดการปัญญาที่วัดด้านสิ่งแวดล้อมเริ่มต้นด้วยข่าวดีดี สี่ พนึก ก. ดึง 437 โรงเรียนในสั

In [11]:
chunks[1]

'โดยเลือกเมนูเครื่องหมายคำถามและพิมพ์ข้อสักถามหรือแสดงความคิดเห็นเข้าไปในระบบ และกดปุ่มเซ็นระบบจะทำการจัดเรียงลำดับก่อนหลัง ตามลำดับเวลาที่มีคำถามหรือข้อคิดเห็นมาทั้งนี้บริษัทขอสงวนสิทธิ์ในการคัดกรองคำถามที่เหมาะสมตามวาราการประชุมกรณีมีคำถามเป็นจำนวนมาก เพื่อรักษาเวลาในการประชุมทางบริษัทจะดำเนินการตอบคำถามของท่านที่เกี่ยวกับวาราการประชุมนั้นๆภายหลังการประชุมเสร็จสิ้นทางอีเมลที่ท่านผู้ถือหุ้นได้โรงทะเบียนไว้ค่ะ เพื่อปฏิบัติให้เป็นเป็นความนโยบายการกํากับดูแลกิจการของบริษัท บริษัทได้จัดให้มีการบันทึกเทปการประชุมและจัดนําขึ้นเว็บไซต์ของบริษัทหลังจากวันประชุมเพื่อให้ผู้ถือหุ้นที่มีได้เข้าร่วมประชุมหรือผู้สนใจทั่วไปได้รับชมกรณีที่ผู้ถือหุ้นประสบปัญหาการเข้าระบบการประชุมผู้ถือหุ้นหรือระบบการออกเสียงลงคะแนนกรณาปฏิบัติตามคู่มือการเข้าร่วมประชุมผู้ถือหุ้นสามัญประจำปี 2566และการออกเสียงลงคะแนนผ่านสื่ออิเล็กทรอนิกส์โดยแอปพลิเคชัน IR Plus AGMที่ได้จัดส่งให้กับผู้ถือหุ้นพร้อมหนังสือเชิญประชุมหรือการุณาติดต่อเจ้าหน้าที่ที่ดูแลตามระบบตามเบอร์โทรศัพท์ที่ได้แจ้งไว้ในคู่มือค่ะ นอกจากนี้เพื่อความโปร่งใสใน